In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv("test.csv")

In [5]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [7]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [8]:
# one-hot 编码
train_d = train_data
dummies = pd.get_dummies(train_data['label'], prefix='label', drop_first=False)
train_features = train_d.drop('label', axis=1)
train_labels = dummies
train_data = pd.concat([train_labels, train_features], axis=1)

test_features = test_data

In [9]:
train_data.head()

,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# 参数
learning_rate = 0.01
training_epochs = 200
# 如果没有足够内存，可以降低 batch size
batch_size = 128  
display_step = 1

n_input = 784
n_classes = 10

# 隐藏层特征数量
n_hidden_layer1 = 256
n_hidden_layer2 = 128


# 层权重和偏置项的储存
weights = {
    'hidden_layer1': tf.Variable(tf.truncated_normal([n_input, n_hidden_layer1])),
    'hidden_layer2': tf.Variable(tf.truncated_normal([n_hidden_layer1, n_hidden_layer2])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_layer2, n_classes]))
}

biases = {
    'hidden_layer1': tf.Variable(tf.random_normal([n_hidden_layer1])),
    'hidden_layer2': tf.Variable(tf.truncated_normal([n_hidden_layer2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# tf Graph input
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, n_classes])

# x_flat = tf.reshape(x, [-1, n_input])

In [11]:
# ReLU作为隐藏层激活函数
layer_1 = tf.add(tf.matmul(x, weights['hidden_layer1']),\
    biases['hidden_layer1'])
layer_1 = tf.nn.relu(layer_1)

layer_2 = tf.add(tf.matmul(layer_1, weights['hidden_layer2']),\
    biases['hidden_layer2'])
layer_2 = tf.nn.relu(layer_2)
# 输出层的线性激活函数
logits = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])

In [12]:
# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)\
    .minimize(cost)
    
# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    
    current_cost = sess.run(
        cost,
        feed_dict={x: last_features, y: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={x: last_features,y: last_labels})
    print('Epoch: {} - Cost: {} Valid Accuracy: {}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))

def normalize_grayscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    a = 0.1
    b = 0.9
    grayscale_min = 0
    grayscale_max = 255
    return a + ( ( (image_data - grayscale_min)*(b - a) )/( grayscale_max - grayscale_min ) )

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        # total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        # for i in range(total_batch):
        batch_x, batch_y = normalize_grayscale(train_features), train_labels
        # Run optimization op (backprop) and cost op (to get loss value)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        print_epoch_stats(epoch, sess, batch_x, batch_y)
    
    predictions = sess.run(logits, feed_dict={x: test_features})
    test_pre = np.argmax(predictions, 1)
    
    conv_y_submission = pd.DataFrame({
        'ImageId': range(1, 28001), 
        'Label': test_pre 
    })

    conv_y_submission.head()
    print(conv_y_submission)
        


Epoch: 0 - Cost: 571.0138549804688 Valid Accuracy: 0.10466666519641876
Epoch: 1 - Cost: 435.8805847167969 Valid Accuracy: 0.12733332812786102
Epoch: 2 - Cost: 334.8888244628906 Valid Accuracy: 0.1612619012594223
Epoch: 3 - Cost: 273.5429382324219 Valid Accuracy: 0.20538094639778137
Epoch: 4 - Cost: 229.50318908691406 Valid Accuracy: 0.21797619760036469
Epoch: 5 - Cost: 200.40830993652344 Valid Accuracy: 0.2768095135688782
Epoch: 6 - Cost: 179.9172821044922 Valid Accuracy: 0.2938571572303772
Epoch: 7 - Cost: 164.9512939453125 Valid Accuracy: 0.3356666564941406
Epoch: 8 - Cost: 152.8709716796875 Valid Accuracy: 0.35545238852500916
Epoch: 9 - Cost: 142.9115447998047 Valid Accuracy: 0.38366666436195374
Epoch: 10 - Cost: 134.47691345214844 Valid Accuracy: 0.4006190598011017
Epoch: 11 - Cost: 127.17005920410156 Valid Accuracy: 0.4215952455997467
Epoch: 12 - Cost: 120.77884674072266 Valid Accuracy: 0.43799999356269836
Epoch: 13 - Cost: 115.12945556640625 Valid Accuracy: 0.453166663646698
Epoc

In [ ]:
conv_y_submission.to_csv('./conv_submission.csv', index=False)